# Interactive Peak Fitting Tool for FIB TOF-SIMS / Mass Spectra

## Overview

This Jupyter Notebook-based tool is designed for interactive peak fitting of FIB TOF-SIMS and other mass spectrometry data. It supports Gaussian, Lorentzian, and PseudoVoigt models with optional baseline correction, smoothing, and default settings on uranium isotope ratio analysis.

**Current Version:** 1.5.7

**Author:** Xiao Sun [https://github.com/xiaosun622]

---

## Workflow: Step-by-Step Procedure

### 1. Load Spectrum File

* Reads a tab-delimited `.txt` file.
* Expected columns:

  * `mass/charge (m/Q)` (x-axis)
  * `Total (cts/TOF-Extraction)` (intensity)

### 2. Apply Smoothing (Optional)

* Savitzky–Golay smoothing filter reduces noise while preserving peak shape.
* Adjustable parameters:

  * `Smooth Win`: Window size
  * `Polyorder`: Polynomial order used in smoothing

### 3. Define Peak Regions

* User inputs peak center and range width (±) for each ion.
* Regions are defined as: `center ± range_width`

### 4. Baseline Correction (if enabled)

* Options:

  * `average`: Flat baseline using predicted intensity ± offset
  * `linear`: Line fit to surrounding regions
  * `polynomial`: 2nd-order polynomial fit
* This background is subtracted from the signal to isolate the peak.

### 5. Select Fit Model

* Choose between symmetric or asymmetric peak shapes:

  * Gaussian
  * Lorentzian
  * PseudoVoigt (or Voigt for asymmetric cases)

### 6. Fit the Peak

* The fitting is applied to the **baseline-corrected signal**.
* Initial parameters are estimated (center, amplitude, sigma).
* The model is optimized to minimize the squared difference between corrected data and prediction.

### 7. Extract Results

* From the fit result:

  * Best-fit curve (`model_prediction`)
  * Fitting Deviation (`data - prediction`)
  * R² (goodness of fit)
  * Area (peak amplitude)

### 8. Plot Outputs

Each subplot includes:

* Smoothed signal
* Corrected signal
* Model fit
* **Fitting Deviation** (formerly “Residuals”)

### 9. Calculate Isotope Ratios

For isotope pairs (e.g. 235U vs 238U), the ratio is:

```math
\text{Ratio} = \frac{\text{Area}_{235}}{\text{Area}_{235} + \text{Area}_{238}}
```

### 10. Save Outputs (Manual Trigger)

* Press "Save Results" after fitting to export:

  * A PNG image of all plots
  * A CSV summary table with areas, R², and isotope ratios

---

## Terminology

| Term                  | Meaning                                                         |
| --------------------- | --------------------------------------------------------------- |
| **Fitting Deviation** | Difference between corrected data and model fit                 |
| **Baseline**          | Estimated background under the peak (subtracted before fitting) |
| **Best Fit**          | Model prediction using optimized parameters                     |
| **R²**                | Coefficient of determination; closer to 1 means better fit      |

---

## Requirements

* Python 3.7+
* `pandas`, `numpy`, `matplotlib`, `scipy`, `lmfit`, `ipywidgets`

Install via pip:

```bash
pip install pandas numpy matplotlib scipy lmfit ipywidgets
```

---

MIT License

Copyright (c) 2025 xiaosun622

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [20]:
%pip install pandas numpy matplotlib scipy lmfit ipywidgets

In [21]:
# === Interactive Peak Fitting Script for TOF-SIMS / Mass Spectra ===
"""
Title: Interactive Peak Fitting Tool
Author: Xiao Sun [https://github.com/xiaosun622]
Version: 1.5.7
Date: 19-06-2025

Description:
This script provides an interactive Jupyter Notebook-based interface for peak fitting in TOF-SIMS or general mass spectrometry spectra.
Users can select fitting ranges, apply smoothing, choose fitting models (Gaussian, Lorentzian, PseudoVoigt), and toggle between symmetric/asymmetric peak models.

Workflow:
1. Load a tab-delimited spectrum file (.txt) with mass/charge and intensity columns.
2. Apply Savitzky–Golay smoothing to reduce noise.
3. Define peak regions interactively via widgets (mass center and range width).
4. For each peak:
   - Perform an initial model fit.
   - Estimate the baseline using fitted values ± a user-defined offset.
   - Subtract the baseline and refit.
   - Extract and report the amplitude (area) of the corrected fit.
5. Plot fitted peaks and calculate isotope ratios.

Interactive Options:
- Fitting model (Gaussian, Lorentzian, PseudoVoigt)
- Symmetric vs. asymmetric fitting
- Smoothing window and polynomial order
- Custom m/z center and range width for each peak
- Use baseline correction or not

Smoothing Guidelines:
# Smooth Win: Number of data points in the window. Larger = smoother, but may distort narrow peaks. Start with 11–19.
# Polyorder: Degree of polynomial fit in each window. Try 2 or 3 for peak preservation.

Fit Quality Assessment:
# A "good fit" is typically indicated by a high R² value close to 1.0 (e.g., > 0.98), minimal residuals, and a visually smooth match between the fit curve and data.
# Poor fits may result from incorrect model choice, too wide or narrow fitting windows, or baseline estimation errors.
"""

# === [IMPORTS] ===
import pandas as pd  # data handling
import numpy as np  # numerical operations
import matplotlib.pyplot as plt  # plotting
from scipy.signal import savgol_filter  # for smoothing spectra
from scipy.integrate import trapezoid  # optional integration method
from lmfit.models import GaussianModel, LorentzianModel, PseudoVoigtModel, VoigtModel  # curve models
import ipywidgets as widgets  # interactive widgets
from IPython.display import display, clear_output  # Jupyter display control
import warnings
import os
import io

# Optional model for skewed peaks if available
try:
    from lmfit.models import SkewedGaussianModel
except ImportError:
    SkewedGaussianModel = None

# Suppress known user warnings
warnings.filterwarnings("ignore", category=UserWarning, module="uncertainties.core")

# === [DATA INIT] ===
x_column = 'mass/charge (m/Q)'
y_column = 'Total (cts/TOF-Extraction)'
txt_file = None  # name of uploaded file
x = y = data = None  # to be populated after upload

# File upload widget
upload_widget = widgets.FileUpload(
    accept='.txt', multiple=False, description='Upload .txt File',
    layout=widgets.Layout(width='300px')
)

# === [WIDGET SETUP] ===
# Fit model dropdown
fit_model = widgets.Dropdown(options=['gaussian', 'lorentzian', 'pseudovoigt'], value='lorentzian', description='Fit Model:')

# Fitting options
symmetric_fit = widgets.Checkbox(value=True, description='Symmetric Peak')
use_baseline = widgets.Checkbox(value=True, description='Baseline Correction')
baseline_type = widgets.Dropdown(options=['average', 'linear', 'polynomial'], value='linear')

# Smoothing sliders
smoothing_window = widgets.IntSlider(value=11, min=3, max=51, step=2, description='Smooth Win:', layout=widgets.Layout(width='300px'))
smoothing_poly = widgets.IntSlider(value=3, min=1, max=5, step=1, description='Polyorder:', layout=widgets.Layout(width='250px'))

# Pixel and Frames
pixel_widget = widgets.IntText(value=256, description='Pixel:', layout=widgets.Layout(width='140px'))
frames_widget = widgets.IntText(value=200, description='Frames:', layout=widgets.Layout(width='140px'))

# Define regions to analyze
region_labels = ['235U', '238U', '235UO', '238UO', '235UO2', '238UO2']
peak_centers = {'235U': 235.0, '238U': 238.0, '235UO': 251.0, '238UO': 254.0, '235UO2': 267.0, '238UO2': 270.0}
range_width_widgets = {}
baseline_offset_widgets = {}

# Create input widgets for each isotope region
for label in region_labels:
    center_widget = widgets.FloatText(value=peak_centers[label], description=f'{label}:', layout=widgets.Layout(width='150px', margin='0 10px 0 0'), step=0.1, format='%.1f')
    width_widget = widgets.FloatText(value=1.0, description="Peak range: ±", layout=widgets.Layout(width='140px', margin='0 10px 0 0'), step=0.1, format='%.1f')
    offset_widget = widgets.FloatText(value=1.0, description="Baseline: ±", layout=widgets.Layout(width='140px', margin='0 10px 0 0'), step=0.05, format='%.1f')
    range_width_widgets[label] = (center_widget, width_widget)
    baseline_offset_widgets[label] = offset_widget

# Action buttons
fit_button = widgets.Button(description="Fit and Calculate Ratios", layout=widgets.Layout(width='300px', height='30px'))
save_button = widgets.Button(description="Save Results", layout=widgets.Layout(width='200px', height='30px'))

# Assemble UI layout
ui = widgets.VBox([
    upload_widget,
    widgets.HBox([fit_model, symmetric_fit, use_baseline, baseline_type]),
    widgets.HBox([smoothing_window, smoothing_poly]),
    widgets.HBox([pixel_widget, frames_widget]),
    widgets.HTML("<b>Edit Peak Centers and Ranges</b><br><i>Baseline estimated from model at center ± baseline m/z offset</i>"),
    *[widgets.HBox([range_width_widgets[label][0], range_width_widgets[label][1], baseline_offset_widgets[label]]) for label in region_labels],
    widgets.HBox([fit_button, save_button])
])

# Global containers for outputs
fig = None
df_summary = None

# === Retrieve selected model from lmfit ===
def get_model(model_type, symmetric=True):
    """Return model instance based on user selection."""
    if model_type == 'gaussian':
        return GaussianModel() if symmetric else (SkewedGaussianModel() if SkewedGaussianModel else GaussianModel())
    elif model_type == 'lorentzian':
        return LorentzianModel() if symmetric else VoigtModel()
    elif model_type == 'pseudovoigt':
        return PseudoVoigtModel() if symmetric else VoigtModel()
    else:
        raise ValueError("Unsupported model type")

# === Fit one peak region with baseline correction ===
def fit_peak_custom_baseline(x_all, y_all, model_type, mz_range, baseline_offset, symmetric, use_baseline, baseline_mode):
    # x_all, y_all: full smoothed spectra arrays
    # mz_range: user-selected m/z range for this peak

    mask_peak = (x_all >= mz_range[0]) & (x_all <= mz_range[1])
    x_peak = x_all[mask_peak]  # m/z values inside the peak window
    y_peak = y_all[mask_peak]  # corresponding intensities

    if len(x_peak) < 5:
        raise ValueError("Peak range too narrow or no data points available.")

    model = get_model(model_type, symmetric=symmetric)  # initialize selected model

    # === Estimate initial parameters ===
    center_guess = (mz_range[0] + mz_range[1]) / 2  # midpoint of selected range
    height_guess = np.max(y_peak)
    sigma_guess = (mz_range[1] - mz_range[0]) / 4
    amplitude_guess = height_guess * sigma_guess * np.sqrt(2 * np.pi)
    params = model.make_params(center=center_guess, amplitude=amplitude_guess, sigma=sigma_guess)

    # === Initial fit without baseline correction ===
    initial_result = model.fit(y_peak, params, x=x_peak)  # returns ModelResult

    # === Estimate baseline and subtract ===
    if use_baseline:
        if baseline_mode == 'average':
            left_val = np.interp(center_guess - baseline_offset, x_peak, initial_result.best_fit)
            right_val = np.interp(center_guess + baseline_offset, x_peak, initial_result.best_fit)
            baseline = np.mean([left_val, right_val])
            y_corr = y_peak - baseline
        elif baseline_mode == 'linear':
            center = (mz_range[0] + mz_range[1]) / 2
            left_mask = (x_all >= center - baseline_offset) & (x_all < center - baseline_offset / 2)
            right_mask = (x_all > center + baseline_offset / 2) & (x_all <= center + baseline_offset)
            x_baseline = np.concatenate([x_all[left_mask], x_all[right_mask]])
            y_baseline = np.concatenate([y_all[left_mask], y_all[right_mask]])
            p = np.polyfit(x_baseline, y_baseline, deg=1)  # linear baseline fit
            baseline_line = np.polyval(p, x_peak)  # evaluate baseline
            y_corr = y_peak - baseline_line
        elif baseline_mode == 'polynomial':
            p = np.polyfit(x_peak, y_peak, deg=2)  # quadratic fit
            baseline_line = np.polyval(p, x_peak)
            y_corr = y_peak - baseline_line
        else:
            y_corr = y_peak
    else:
        y_corr = y_peak

    # === Final model fit on baseline-corrected data ===
    final_result = model.fit(y_corr, model.make_params(center=center_guess, amplitude=amplitude_guess, sigma=sigma_guess), x=x_peak)

    # === Output results ===
    area = final_result.params['amplitude'].value  # area under peak
    residuals = y_corr - final_result.best_fit
    ss_res = np.sum(residuals**2)  # sum of squared residuals
    ss_tot = np.sum((y_corr - np.mean(y_corr))**2)
    r_squared = 1 - ss_res / ss_tot if ss_tot > 0 else np.nan  # fit quality

    return final_result, x_peak, y_peak, y_corr, area, residuals, r_squared

def run_fitting(b):
    global df_summary, fig, x, y
    clear_output(wait=True)
    display(ui)

    if x is None or y is None:
        print("Please upload a .txt file before running fitting.")
        return

    if smoothing_window.value >= len(y):
        print("Error: Smoothing window too large for data length.")
        return

    y_smooth = savgol_filter(y, smoothing_window.value, smoothing_poly.value)

    regions = {
        label: (
            range_width_widgets[label][0].value - range_width_widgets[label][1].value,
            range_width_widgets[label][0].value + range_width_widgets[label][1].value
        )
        for label in region_labels
    }
    offsets = {label: baseline_offset_widgets[label].value for label in region_labels}
    ratio_groups = [('235U', '238U'), ('235UO', '238UO'), ('235UO2', '238UO2')]

    areas = {}
    r2_values = {}
    results = {}

    fig, axes = plt.subplots(3, 2, figsize=(14, 12))
    axes = axes.flatten()

    for i, label in enumerate(region_labels):
        mz_range = regions[label]
        baseline_offset = offsets[label]

        try:
            result, x_fit, y_fit, y_corr, area, residuals, r2 = fit_peak_custom_baseline(
                x, y_smooth, fit_model.value, mz_range, baseline_offset, symmetric_fit.value, use_baseline.value, baseline_type.value
            )
        except Exception as e:
            print(f"Error fitting {label}: {e}")
            continue

        areas[label] = area
        r2_values[label] = r2
        results[label] = result

        ax = axes[i]
        ax.plot(x_fit, y_fit, label='Smoothed', color='blue')
        if use_baseline.value:
            ax.plot(x_fit, y_corr, label='Corrected', color='purple')
        ax.plot(x_fit, result.best_fit, 'r--', label='Fit')
        ax.plot(x_fit, residuals, 'k:', label='Fitting deviation')
        ax.set_title(f"{label} (R² = {r2:.4f})")
        ax.legend()
        ax.grid(True)

    plt.tight_layout()
    plt.show()

    def calc_ratio(a1, a2):
        return a1 / (a1 + a2) if (a1 + a2) > 0 else np.nan

    summary_rows = []
    model_name = fit_model.value.capitalize()

    for label in region_labels:
        area_val = areas.get(label, np.nan)
        summary_rows.append({
        "Ions": label,
        "Area": area_val,
        "R²": r2_values.get(label, np.nan),
        "235U Isotopic Ratio": ""
    })

    for a1, a2 in ratio_groups:
        r = calc_ratio(areas.get(a1, 0), areas.get(a2, 0))
        for row in summary_rows:
            if row["Ions"] == a1:
                row["235U Isotopic Ratio"] = "" if pd.isna(r) else f"{r:.4f}"

    df_summary = pd.DataFrame(summary_rows, columns=["Ions", "Area", "R²", "235U Isotopic Ratio"])
    pixel = pixel_widget.value
    frames = frames_widget.value
    df_summary.insert(1, "Counts", df_summary["Area"] * frames * (pixel ** 2))
    df_summary = df_summary.drop(columns=["Area"])
    df_summary.index = np.arange(1, len(df_summary) + 1)
    display(df_summary)

def save_results(b):
    global df_summary, fig
    if df_summary is None or fig is None:
        print("No results to save. Please run the fitting first.")
        return
    base_name = os.path.splitext(os.path.basename(txt_file))[0] if txt_file else "output"
    fig.savefig(f"{base_name}_fit.png", dpi=300)

    # Prepare fitting metadata
    settings = {
        "Fitting Model": fit_model.value,
        "Symmetric Peak": symmetric_fit.value,
        "Baseline Correction": use_baseline.value,
        "Baseline Type": baseline_type.value,
        "Smoothing Window": smoothing_window.value,
        "Smoothing Polyorder": smoothing_poly.value
    }

    # Construct metadata as header lines
    header_lines = ["Fitting Parameters Used:"]
    header_lines += [f" {k}: {v}" for k, v in settings.items()]
    header_lines += [
        f" Pixel: {pixel_widget.value}",
        f" Frames: {frames_widget.value}"]

    header_lines += ["Peak Regions (m/z range):"]
    for label in region_labels:
        center = range_width_widgets[label][0].value
        width = range_width_widgets[label][1].value
        offset = baseline_offset_widgets[label].value
        header_lines.append(f" {label}: center = {center}, ±range = {width}, ±baseline = {offset}")

    csv_path = f"{base_name}_summary.csv"
    with open(csv_path, "w", encoding="utf-8") as f:
        df_summary.to_csv(f, index=False)  # Write the results table
        f.write("\n")  # Blank line before metadata section

    with open(csv_path, "a", encoding="utf-8") as f:
        for line in header_lines:
                        f.write(line.strip() + "\n")  # Each metadata line on a new row

    print(f"Saved: {base_name}_fit.png and {base_name}_summary.csv with fitting settings")
    print(f"Saved: {base_name}_fit.png and {base_name}_summary.csv")

# === [EVENT BINDING] ===
fit_button.on_click(run_fitting)
save_button.on_click(save_results)

# === [UPLOAD EVENT HANDLER] ===
def handle_upload(change):
    global data, x, y, txt_file
    uploaded = upload_widget.value
    if not uploaded:
        print("No file uploaded.")
        return
    file_info = list(uploaded.values())[0] if isinstance(uploaded, dict) else uploaded[0]
    txt_file = file_info['name']
    content = file_info['content']
    try:
        df = pd.read_csv(io.BytesIO(content), sep='\t', comment='#')
        df = df.dropna(subset=[x_column, y_column])
        data = df
        x = data[x_column].values
        y = data[y_column].values
        clear_output(wait=True)
        display(ui)
        print(f"Loaded: {txt_file} | {len(data)} rows")
    except Exception as e:
        print(f"Failed to load file: {e}")

upload_widget.observe(handle_upload, names='value')

# === [DISPLAY UI INITIALLY] ===
display(ui)
